In [1]:
%%capture
!pip install mysql-connector-python pandas scikit-learn

In [2]:
import mysql.connector
import pandas as pd

In [3]:
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="95095",
    database="telecom"
)

query = 'Select * from customer_churn'
df = pd.read_sql(query, conn)

conn.close()

C:\Users\mayan\AppData\Local\Temp\ipykernel_4404\2284037864.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


In [4]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,estimated_revenue,likely_complaint,tenure_segment
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,29.85,1,New
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.50,No,1936.30,0,Long-Term
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,107.70,0,New
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,1903.50,0,Long-Term
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,141.40,1,New


In [5]:
# Checking The Missing Values.
df.isnull().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges         0
Churn                0
estimated_revenue    0
likely_complaint     0
tenure_segment       0
dtype: int64

In [6]:
# Drop CustomerID (Non-Numeric)
df.drop('customerID', axis=1, inplace=True)

In [7]:
# Convert TotalCharges To Numeric (Some May Be Missing)
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

In [8]:
# Drop Missing Values If Any
df.dropna(inplace=True)

In [9]:
# Convert Target Column
df['Churn'] = df['Churn'].map({'Yes' : 1, 'No' : 0})

In [10]:
# One-Hot Encode Categorical Variables
df_encoded = pd.get_dummies(df, drop_first=True)

In [11]:
# Trainging Testing Split
from sklearn.model_selection import train_test_split

# Feature & Target
X = df_encoded.drop('Churn', axis=1)
y = df_encoded['Churn']

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Tarin Model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Logistic Regression : ")
print(classification_report(y_test, y_pred))

Logistic Regression : 
              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1033
           1       0.62      0.47      0.53       374

    accuracy                           0.78      1407
   macro avg       0.72      0.68      0.70      1407
weighted avg       0.77      0.78      0.77      1407



C:\Users\mayan\AppData\Roaming\Python\Python312\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [12]:
# Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Random Forest : ")
print(classification_report(y_test, y_pred))

Random Forest : 
              precision    recall  f1-score   support

           0       0.82      0.90      0.86      1033
           1       0.64      0.47      0.54       374

    accuracy                           0.79      1407
   macro avg       0.73      0.68      0.70      1407
weighted avg       0.77      0.79      0.78      1407



In [13]:
# XGBoost/Gradient Boosting Classifier
from sklearn.ensemble import GradientBoostingClassifier

model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("XGBoost : ")
print(classification_report(y_test, y_pred))

XGBoost : 
              precision    recall  f1-score   support

           0       0.83      0.90      0.86      1033
           1       0.64      0.49      0.56       374

    accuracy                           0.79      1407
   macro avg       0.73      0.70      0.71      1407
weighted avg       0.78      0.79      0.78      1407



In [14]:
# Support Vector Classifier
from sklearn.svm import SVC

model = SVC(probability=True)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("SVC : ")
print(classification_report(y_test, y_pred))

SVC : 
              precision    recall  f1-score   support

           0       0.73      1.00      0.85      1033
           1       0.00      0.00      0.00       374

    accuracy                           0.73      1407
   macro avg       0.37      0.50      0.42      1407
weighted avg       0.54      0.73      0.62      1407



C:\Users\mayan\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mayan\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\mayan\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [15]:
# KNeighbors Classifier
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("KNN : ")
print(classification_report(y_test, y_pred))

KNN : 
              precision    recall  f1-score   support

           0       0.81      0.89      0.85      1033
           1       0.59      0.41      0.48       374

    accuracy                           0.77      1407
   macro avg       0.70      0.65      0.67      1407
weighted avg       0.75      0.77      0.75      1407



In [16]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Naive Bayes : ")
print(classification_report(y_test, y_pred))

Naive Bayes : 
              precision    recall  f1-score   support

           0       0.92      0.62      0.74      1033
           1       0.45      0.85      0.59       374

    accuracy                           0.68      1407
   macro avg       0.68      0.73      0.66      1407
weighted avg       0.79      0.68      0.70      1407



# Note :- Naive Bayes is better at identifying churners, because:

#  Recall (0.85) is very high — it correctly identifies 85% of churners

#  F1 Score (0.59) for churn is higher than Random Forest (0.54)

In [17]:
%%capture
# Install ELI5
!pip install eli5

In [19]:
import eli5
from eli5.sklearn import PermutationImportance
from sklearn.ensemble import RandomForestClassifier


rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)


perm = PermutationImportance(rf,random_state=42).fit(X_test, y_test)


eli5.show_weights(perm, feature_names = X.columns.tolist())

Weight,Feature
0.0108 ± 0.0056,PaperlessBilling_Yes
0.0104 ± 0.0084,InternetService_Fiber optic
0.0065 ± 0.0040,Contract_One year
0.0060 ± 0.0087,tenure
0.0057 ± 0.0060,TotalCharges
0.0051 ± 0.0052,MultipleLines_Yes
0.0034 ± 0.0072,MonthlyCharges
0.0023 ± 0.0059,Contract_Two year
0.0017 ± 0.0014,OnlineBackup_No internet service
0.0016 ± 0.0080,estimated_revenue
